In [1]:
import sys, os

from tqdm import tqdm
tqdm.pandas()

from dotenv import load_dotenv
import pandas as pd

from langchain.text_splitter import RecursiveCharacterTextSplitter

load_dotenv("../.env")

True

In [2]:
df = pd.read_csv("../data/bigbasket_garnier.csv")
df.head(2)

,index,product,category,sub_category,brand,sale_price,market_price,type,rating,description,questions
0,103,Fructis Serum - Long & Strong,Beauty & Hygiene,Hair Care,Garnier,230.0,230.0,Shampoo & Conditioner,4.8,Garnier Fruits Long & Strong Strengthening Ser...,['What are the key ingredients in Fructis Seru...
1,163,Color Naturals Creme Riche Ultra Hair Color - ...,Beauty & Hygiene,Hair Care,Garnier,199.0,199.0,Hair Color,4.1,Garnier Color Naturals is creme hair colour wh...,['How long does Garnier Color Naturals Creme R...


In [3]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=1.0)

In [4]:
from langchain_core.messages import HumanMessage

def get_questions(product_name, product_description):
    questions = chat.invoke(
    [
        HumanMessage(
            content=f"""Your job is to generate questions for the product descriptions such that it is hard to answer the question.


Example 1
Product name: Fructis Serum - Long & Strong
Garnier Fruits Long & Strong Strengthening Serum detangles unruly hair, softens hair without heaviness and helps stop split and breakage ends. It is enriched with the goodness of Grape seed and avocado oil that result in smoother, shinier and longer hair.  
Questions: 
- Does Garnier Fruits Long & Strong Strengthening Serum help in hair growth?
- Which products contain avocado oil?

Example 2
Product name: Color Naturals Creme Riche Ultra Hair Color - Raspberry Red
Garnier Color Naturals is creme hair colour which gives 100% Grey Coverage and ultra visible colour with 50% more shine. It has a superior Colour Lock technology which gives you a rich long-lasting colour that lasts up to 8 weeks. Color Naturals comes in a range of 8 gorgeous shades especially suited for Indian skin tones. It is available in an easy to use kit which can be used at your convenience in the comfort of your house! It is enriched with the goodness of 3 oils - Almond, Olive and Avocado which nourishes hair and provides shiny, long-lasting colour. Your hair will love the nourishment and you will love the colour!
Questions: 
- How long does Color Naturals Creme Riche Ultra Hair Color last?
- Which product for hair color is suited for indian skin?
- How many colors are available in Color Naturals Hair Color?

Product name: Black Naturals Hair Colour Shade 1-Deep Black 20 ml + 20 gm
It is an oil-enriched cream colour which gives natural-looking black hair. Works in 15 minutes, non-drip cream. Maintains softness, smoothness, and shine. No ammonia hair colour. Lasts for 6 weeks.
Questions: 
- Does Black Naturals Hair Colour contain ammonia?
- How muct time do you have to keep Grey Naturals Hair Colour on your hair?

Now generate 3 questions which can be answered and 2 questions which cannot be answered for this product based on description. For answerable questions use the exact product name in the questions. For unanswerable questions, either change product name or change product info. There should not be duplicates in the 5 questions. Return questions starting with - instead of numbers.

Product name: {product_name}
{product_description}
Questions: """
        )
    ]
)
    questions = questions.content.replace("- ", "").split("\n")
    questions = list(filter(None, questions)) 
    return questions

product_name, product_description = df.iloc[10][["product", "description"]]
print(product_name, "\n", product_description)
get_questions(product_name, product_description)

Skin Naturals - Light Complete White Face Wash 
 Garnier White Complete is a women's face wash enriched with Yuzu Lemon essence. Yuzu Lemon is known as the King of Lemons since it contains a high concentration of Vitamin C and is known for its strong brightening, exfoliating and anti-oxidant properties. This daily face wash for women thoroughly cleanses, clarifies and brightens the skin. It helps recover a clear, radiant and even complexion. It is suitable for all skin types.


['How does Skin Naturals Light Complete White Face Wash help in skin brightening?',
 'What is the main ingredient in Skin Naturals Light Complete White Face Wash?',
 'How does Skin Naturals Light Complete White Face Wash help in skin exfoliation?',
 'What are the benefits of Yuzu Lemon in a face wash?',
 'Can Skin Naturals Light Complete White Face Wash be used on sensitive skin?']

In [ ]:
df["questions"] = df.progress_apply(lambda x: get_questions(x["product"], x["description"]), axis=1)

In [ ]:
df.head()

In [ ]:
df.to_csv("../data/bigbasket_garnier.csv", index=False)